# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [2]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [3]:
# Run this code:
salaries = pd.read_csv(r'..\data\Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [4]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [5]:
from scipy.stats import ttest_1samp

hourly_workers = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Hourly Rate'].notna())]
hourly_rates = hourly_workers['Hourly Rate'].values
t_stat, p_val = ttest_ind(hourly_rates, 30)

print(f'T-statistic: {t_stat}')
print(f'P-value: {p_val}')

alpha = 0.05  

if p_val < alpha:
    print("La hipótesis de que el salario por hora es diferente de $30/hora se rechaza a un nivel de significancia del 5%.")
else:
    print("La hipótesis de que el salario por hora es diferente de $30/hora no se rechaza a un nivel de significancia del 5%.")


T-statistic: 0.23020575383291658
P-value: 0.8179377615647019
La hipótesis de que el salario por hora es diferente de $30/hora no se rechaza a un nivel de significancia del 5%.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [8]:
from scipy import stats

mean_salary = np.mean(hourly_rates)
std_error = stats.sem(hourly_rates)
n = len(hourly_rates)
df = n - 1
confidence = 0.95
ci = stats.t.interval(confidence, df, loc=mean_salary, scale=std_error)


print(f"Media de la muestra: {mean_salary:.2f}")
print(f"Error estándar: {std_error:.2f}")
print(f"Intervalo de confianza del {confidence*100:.0f}%: ({ci[0]:.2f}, {ci[1]:.2f})")

Media de la muestra: 32.79
Error estándar: 0.14
Intervalo de confianza del 95%: (32.52, 33.05)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [10]:
from statsmodels.stats.proportion import proportions_ztest

num_hourly_workers = len(hourly_workers)
num_total_workers = len(salaries)
proportion_null = 0.25

count = num_hourly_workers
nobs = num_total_workers
stat, p_value = proportions_ztest(count, nobs, proportion_null)

print(f"Número de trabajadores por hora: {num_hourly_workers}")
print(f"Número total de trabajadores: {num_total_workers}")
print(f"Proporción de trabajadores por hora observada: {num_hourly_workers / num_total_workers:.2f}")
print(f"Estadístico z: {stat:.4f}")
print(f"P-valor: {p_value:.4f}")

alpha = 0.05  
if p_value < alpha:
    print("Rechazamos la hipótesis nula. La proporción de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("No rechazamos la hipótesis nula. La proporción de trabajadores por hora no es significativamente diferente del 25%.")

Número de trabajadores por hora: 8022
Número total de trabajadores: 33183
Proporción de trabajadores por hora observada: 0.24
Estadístico z: -3.5100
P-valor: 0.0004
Rechazamos la hipótesis nula. La proporción de trabajadores por hora es significativamente diferente del 25%.
